# Some traffic prediction
variation by Artem Ostrovsky to the original notebook created by Anvar Kurmukov,
updated by Bogdan Kirillov, Hekmat Taherinejad, Satyarth Mishra Sharma

---

Try to implement your knowledge in Python functions, classes and debugging:

1. Make DataAccessor.py -> class that creates handy interface to all your data
2. Make Features.py -> class that creates some new artifitial features
3. Add try/exept
4. Run the pipeline using pdb

---

Some links:

pdb https://habr.com/ru/post/104086/, https://pythonim.ru/moduli/pdb-python

classes https://www.w3schools.com/python/python_classes.asp

---

We are using a public dataset compiling weather information and traffic data continuously monitored in the Twin Cities, Minnesota from 2012 to 2018. The dataset page can be found [here](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume).

You need to download `Metro_Interstate_Traffic_Volume.csv` and place it in the same directory as this notebook.


In [1]:
import numpy as np
import pandas as pd

# 1. Loading data

As always in Data Science you are starting with making nice cup of tea (or coffee). Your next move is to load the data:

- Start with loading `Metro_Interstate_Traffic_Volume.csv` file using `pd.read_csv()` function.
- You may also want to increase maximal displayed pandas columns: set `pd.options.display.max_columns` to 30
- Print top 10 observations in the table. `.head()`
- Print last 10 observations in the table. `.tail()`
- Print all the data columns names using method `.columns`
- Print data size (number of rows and columns). This is the `.shape` of the data.

*Almost* every python has a `head` and a `tail` just as DataFrames do.

If you are using Google Colab, you can upload the file in the cell below. If you are NOT using Colab, set COLAB_P in the cell below to False.

In [2]:
# Load the data
data = pd.read_csv('Metro_Interstate_Traffic_Volume.csv')
data

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40.0,Clouds,scattered clouds,2012-10-02 09:00:00,5545.0
1,None,289.36,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 10:00:00,4516.0
2,None,289.58,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 11:00:00,4767.0
3,None,290.13,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 12:00:00,5026.0
4,None,291.14,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 13:00:00,4918.0
...,...,...,...,...,...,...,...,...,...
48199,None,283.45,0.0,0.0,75.0,Clouds,broken clouds,2018-09-30 19:00:00,3543.0
48200,None,282.76,0.0,0.0,90.0,Clouds,overcast clouds,2018-09-30 20:00:00,2781.0
48201,None,282.73,0.0,0.0,90.0,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159.0
48202,None,282.09,0.0,0.0,90.0,Clouds,overcast clouds,2018-09-30 22:00:00,1450.0


In [3]:
# Observe top 10 observations (int)
data.head(10)

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40.0,Clouds,scattered clouds,2012-10-02 09:00:00,5545.0
1,None,289.36,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 10:00:00,4516.0
2,None,289.58,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 11:00:00,4767.0
3,None,290.13,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 12:00:00,5026.0
4,None,291.14,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 13:00:00,4918.0
5,None,291.72,0.0,0.0,1.0,Clear,sky is clear,2012-10-02 14:00:00,5181.0
6,None,293.17,0.0,0.0,1.0,Clear,sky is clear,2012-10-02 15:00:00,5584.0
7,None,293.86,0.0,0.0,1.0,Clear,sky is clear,2012-10-02 16:00:00,6015.0
8,None,294.14,0.0,0.0,20.0,Clouds,few clouds,2012-10-02 17:00:00,5791.0
9,None,293.10,0.0,0.0,20.0,Clouds,few clouds,2012-10-02 18:00:00,4770.0


In [4]:
# Observe last 10 observations (int)
data.tail(10)

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
48194,None,283.84,0.00,0.0,75.0,Rain,proximity shower rain,2018-09-30 15:00:00,4302.0
48195,None,283.84,0.00,0.0,75.0,Drizzle,light intensity drizzle,2018-09-30 15:00:00,4302.0
48196,None,284.38,0.00,0.0,75.0,Rain,light rain,2018-09-30 16:00:00,4283.0
48197,None,284.79,0.00,0.0,75.0,Clouds,broken clouds,2018-09-30 17:00:00,4132.0
48198,None,284.20,0.25,0.0,75.0,Rain,light rain,2018-09-30 18:00:00,3947.0
48199,None,283.45,0.00,0.0,75.0,Clouds,broken clouds,2018-09-30 19:00:00,3543.0
48200,None,282.76,0.00,0.0,90.0,Clouds,overcast clouds,2018-09-30 20:00:00,2781.0
48201,None,282.73,0.00,0.0,90.0,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159.0
48202,None,282.09,0.00,0.0,90.0,Clouds,overcast clouds,2018-09-30 22:00:00,1450.0
48203,None,282.12,0.00,0.0,90.0,Clouds,overcast clouds,2018-09-30 23:00:00,954.0


In [5]:
# Print all the columns/features names (int)
data.columns

Index(['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
       'weather_description', 'date_time', 'traffic_volume'],
      dtype='object')

In [ ]:
# Q1.1 How many columns end with a vowel?
# Q1.2 How many columns start with a vowel?
# Q1.3 Which columns are associated with the condition of weather?
# Q1.4 How many columns have `th` in their names?


### Q1.1

In [10]:
c = 0
for i in data.columns:
    
    if i[-1].lower() in 'aoeiy':
        c += 1
    
print(c)

3


### Q1.2

In [11]:
c = 0
for i in data.columns:
    
    if i[0].lower() in 'aoeiy':
        c += 1
    
print(c)

0


### Q1.3

In [12]:
for i in data.columns:
    if 'weather' in i:
        print(i)

weather_main
weather_description


### Q1.4

In [15]:
c = 0 
for i in data.columns:
    if 'th' in i:
        c += 1
print(c)    

2


In [ ]:
# Print data size (int)

# Q2.1 How many observations are in the data?
# Q2.2 How many features are in the data?


### Q2.1

In [18]:
data.shape[0]

48204

### Q2.2

In [19]:
data.shape[1]

9

# 2. Basic data exploration

Lets do some basics:

`.count()` number of not NaN's in every column.
    
Is there any missing values in the data?     
Count number of unique values in every column .nunique().    
What does this tells you about the features, which are most likely categorical and which are most likely numerical?    
Use pandas `.describe()` to display basic statistic about the data.   
Use pandas `.value_counts()` to count number of unique values in a specific column.   
Use pandas `.min()`, `.max()`, `.mean()`, `.std()` to display specific statistics about the data.    
Use pandas `.dtypes` field to display data types in columns. 
Hint You could use `.sort_index()` or `.sort_values()` to sort the result of `.value_counts()`


In [ ]:
# Display number of not NaN's in every column (int)

# Q3.1 How many NA values are in the `clouds_all` column?
# Q3.2 How many NA values are in the `temp` column?
# Q3.3 How many NA values are in the `rain_1h` column?
# Q3.4 How many NA values are in the `snow_1h` column?
# Q3.5 How many explicit NA values are in the `traffic_volume` column?


### Q3

In [23]:
data.columns

Index(['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
       'weather_description', 'date_time', 'traffic_volume'],
      dtype='object')

In [32]:
data[data.isna()].count()

holiday                0
temp                   0
rain_1h                0
snow_1h                0
clouds_all             0
weather_main           0
weather_description    0
date_time              0
traffic_volume         0
dtype: int64

In [37]:
data.isna()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
48199,False,False,False,False,False,False,False,False,False
48200,False,False,False,False,False,False,False,False,False
48201,False,False,False,False,False,False,False,False,False
48202,False,False,False,False,False,False,False,False,False


In [34]:
data.value_counts()

holiday               temp     rain_1h  snow_1h  clouds_all  weather_main  weather_description  date_time            traffic_volume
None                  294.520  0.0      0.0      1.0         Clear         sky is clear         2017-06-30 11:00:00  4725.0            2
                      295.010  0.0      0.0      40.0        Clouds        scattered clouds     2017-06-21 11:00:00  4808.0            2
                      296.950  0.0      0.0      1.0         Thunderstorm  thunderstorm         2017-06-30 13:00:00  5263.0            2
                      288.890  0.0      0.0      1.0         Clear         sky is clear         2017-09-29 19:00:00  4498.0            2
                      254.220  0.0      0.0      1.0         Clear         sky is clear         2016-12-19 00:00:00  420.0             2
                                                                                                                                      ..
                      274.820  0.0      0.0   

In [35]:
data.describe()

,temp,rain_1h,snow_1h,clouds_all,traffic_volume
count,48203.000000,48203.000000,48204.000000,48201.000000,48199.000000
mean,281.205431,0.334266,0.000222,49.364847,3259.882300
std,13.338022,44.789598,0.008168,39.015492,1986.873655
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,272.160000,0.000000,0.000000,1.000000,1193.000000
50%,282.450000,0.000000,0.000000,64.000000,3380.000000
75%,291.806000,0.000000,0.000000,90.000000,4933.000000
max,310.070000,9831.300000,0.510000,100.000000,7280.000000


### Q3.5

In [ ]:
# Now drop rows with NaN with `.dropna`. Remeber to either reassign your dataframe or provide `inplace=True` argument.


In [38]:
data.dropna(inplace=True)

In [40]:
data.value_counts()

holiday               temp     rain_1h  snow_1h  clouds_all  weather_main  weather_description  date_time            traffic_volume
None                  294.520  0.0      0.0      1.0         Clear         sky is clear         2017-06-30 11:00:00  4725.0            2
                      295.010  0.0      0.0      40.0        Clouds        scattered clouds     2017-06-21 11:00:00  4808.0            2
                      296.950  0.0      0.0      1.0         Thunderstorm  thunderstorm         2017-06-30 13:00:00  5263.0            2
                      288.890  0.0      0.0      1.0         Clear         sky is clear         2017-09-29 19:00:00  4498.0            2
                      254.220  0.0      0.0      1.0         Clear         sky is clear         2016-12-19 00:00:00  420.0             2
                                                                                                                                      ..
                      274.820  0.0      0.0   

In [41]:
# Display basic data statistics using .describe()
data.describe()

,temp,rain_1h,snow_1h,clouds_all,traffic_volume
count,48190.000000,48190.000000,48190.000000,48190.000000,48190.000000
mean,281.201366,0.334356,0.000222,49.369267,3259.859079
std,13.337406,44.795638,0.008169,39.016127,1986.972809
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,272.160000,0.000000,0.000000,1.000000,1192.250000
50%,282.440000,0.000000,0.000000,64.000000,3380.000000
75%,291.800000,0.000000,0.000000,90.000000,4933.000000
max,310.070000,9831.300000,0.510000,100.000000,7280.000000


In [51]:
# Count number of unique values in every column (int)
a = 0
for i in data.columns:
    print(f'{i} -- {len(data[i].unique())}')
    a +=  len(data[i].unique())  
print(f'Total -- {a}')
# Q4.1 How many unique values are in the `clouds_all` column
# Q4.2 How many unique values are in the `weather_main` column?
# Q4.3 How many unique values are in the `weather_description` column?
# Q4.4 How many unique values are in the `snow_1h` column?
# Q4.5 How many unique values are in the `rain_1h` column?


holiday -- 12
temp -- 5843
rain_1h -- 372
snow_1h -- 12
clouds_all -- 60
weather_main -- 11
weather_description -- 38
date_time -- 40562
traffic_volume -- 6704
Total -- 53614


In [ ]:
# Count frequency of the values in different columns (list of ints in ascending order)
# You could select a column using same syntax as for selecting a key from a dictionary: `data[colname]`
# numpy's `unique` function can be useful for this task

# Q5.1 For every unique `weather_main` value give its number of occurences.
# Q5.2 For every unique `weather_description` value give its number of occurences.


In [ ]:
# Display some column statistics (list of floats, rounded up to 3 digits, e.g. 1.234)

# Q6.1 What are the max, min, mean and the std of the `traffic_volume` column?
# Q6.2 What are the max, min, mean and the std of the `clouds_all` column?
# Q6.3 What are the max, min, mean and the std of the `temp` column?
# Q6.4 What are the max, min, mean and the std of the `rain_1h` column?
# Q6.5 What are the max, min, mean and the std of the `snow_1h` column?


In [ ]:
# Display data types of all columns (int)

# Q7.1 How many columns have `object` data type?
# Q7.2 How many columns have `int64` data type?
# Q7.3 How many columns have `float64` data type?
# Q7.4 What are the columns with dtype == `float64`?
# Q7.5 What are the columns with dtype == `int64`?


# 3. Data selection

In pandas.DataFrame you could select

  Row/s by position (integer number [0 .. number of rows - 1]) .iloc or by DataFrame.index .loc:   

```
  data.loc[0]  
  data.loc[5:10]  
  data.iloc[0]  
  data.iloc[5:10]   
```

Though, this is probably the worst way to manipulate rows.   
  Columns by name

```
  data[columname]
```

  Row/s and columns

```
  data.loc[10, columname]  
  data.iloc[10, columname]  
```

Using boolean mask

```
  mask = data[columname] > value  
  data[mask]  
```

You could combine multiple conditions using & or | (and, or)   

```
cond1 = data[columname1] > value1  
cond2 = data[columname2] > value2  
data[cond1 & cond2]  
```

Using queries .query():  

```
value = 5 
data.query("columname > value")  
```

You could combine multiple conditions using and, or  

```
data.query("(columname1 > value1) and (columname2 > value2)")
```

and others. See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html for more examples.

Remember to use different quotation marks " or ' for columnname inside a query.


In [ ]:
# Select rows by position (int) 

# Q8.1 What is the temperature of the time slot with index 777?
# Q8.2 What is the weather description of the time slot with index 999?
# Q8.3 How much is cloud coverage with index 1337?
# Q8.4 What is the weather main of the time slot with index 314?
# Q8.5 When was the time slot with index of 2718 observed?


In [ ]:
# Select rows by index (int)

# Q9.1 What is the temperature of the time slot on index 1102?
# Q9.2 What is the weather description of the time slot on index 5695?
# Q9.3 How much is cloud coverage on the index 1045?
# Q9.4 What is the weather main of the time slot from index 252?
# Q9.5 When was the time slot with index of 38 captured?


In [ ]:
# Using mask or .query syntax select rows/columns (int)

# Q10.1 How many time slots have less than 270 temperature?
# Q10.2 When was the first "light intensity drizzle" in weather description captured?
# Q10.3 How many time slots have cloud coverage more than 75?
# Q10.4 How many time slots are foggy? (weather_main = Fog)
# Q10.5 When was the last observed timeslot with weather_description "heavy snow"?


In [ ]:
# Q11.1 What is the traffic volume of November 20th 2016, at 20:00?
# Q11.2 What is the amount of rain in the 70th rainy time slot (non-zero rain) of the dataset?
# Q11.3 How much cloud coverage percentage were in sky on October 16th 2012 at 19:00?
# Q11.4 What is the `traffic_volume` of a thirty fourth sample with `clouds_all` == 90?
# Q11.5 What is the "weather_description" in the 20th "weather_main" with Thunderstorm?


In [ ]:
# Q12.1 What is the temperature of the tenth holiday?
# Q12.2 What is the traffic volume for 99-th time slot with cloud coverage 75 percent?
# Q12.3 How much is the temperature of the twelfth holiday?
# Q12.4 How much is the temperature the 666-th time slot with weather_description 'proximity thunderstorm'?
# Q12.5 What is the temperature of 1337-th time slot with clear sky (clouds_all <= 20)?


# 4. Creating new columns

Creating new column of pandas.DataFrame is as easy as:
```
data['new_awesome_column'] = [] 
```
that's it. But such a column is relatively useless. Typically, you would compute something new based on existing data and save it in a new column. For example one might want to sum a number of existing columns:
```
data['sum'] = data[col1] + data[col2] + ...
```
Pandas also provides another powerfull tool: .apply, .map(), .applymap() methods (they are kinda the same, but not quite). https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas . They allow you to apply some function to every value in the column/s (row-wise) or row (column-wise) or cell (element-wise). For example, same computations of sum using .apply():
```
data['sum'] = data[[col1, col2, col3]].apply(sum, axis=1)
```
you are not restricted to existent functions, .apply() accepts any function (including lambda functions):
```
data['sum'] = data[[col1, col2, col3]].apply(lambda x: x[0]+x[1]+x[2], axis=1)
```
or ordinary python function (if this it should have complex behaviour):
```
def _sum(x):
    total = 0
    for elem in x:
        total += elem
    return total

data['sum'] = data[[col1, col2, col3]].apply(_sum, axis=1) 
```
Many pandas methods has axis parameter axis=0 refers to rows, axis=1 refers to columns.

Warning. You should never use for loops to sum numerical elements from the container.

In [ ]:
# Create new columns using the old ones (new column in your DataFrame)

# Q13.1 Create a `temp_in_celcius` column from the existing `temp` (kelvin) using any method above
# Q13.2 Create a new bool column `hot` which indicates whether the time slot was hot (temp > 300)
# Q13.3 Create a new bool column `rainy_and_cloudy` which indicates whether it was rainy (>0.1) AND cloudy (>50)
# Q13.4 Create a new bool column `is_holiday` which indicates whether the day of the time slot falls on any holiday
# Q13.5 Create a new column `traffic_cat` by splitting a `traffic_volume` into 5 ([1..5]) distinct intervals: 0 < x <=20%,
# 20% < x <= 40%, ... 80% < x <= 100% percentiles. You could use `.quantile()` to compute percentiles.


In [ ]:
# Using mask or .query syntax select rows/columns (int).
# For working with dates, define helper functions that operate on the date_time string.

# Q14.1 How many cloudy time slots were captured in autumn 2016? Including both start and end day.
# Q14.2 How many rainy time slots that were captured in the fall, with traffic volume more than 2000?
# Q14.3 How many time slots that are warmer than 270, have weather main "Clouds"?
# Q14.4 What is the minimum traffic volume of time slots captured on March 8th (all years), that was warmer than 290?
# Q14.5 How much is the maximum traffic volume for the time slots were captured in June 2017 and has clear sky (weather_main)?


In [ ]:
# Using mask or .query syntax select rows/columns and compute simple statistics (float)

# Q15.1 What was the average temperature of time slots with main weather "Haze"?
# Q15.2 What was the traffic volume of the coldest time slot of the year 2016?
# Q15.3 What was the traffic volume of the highest amount of snow in one hour?
# Q15.4 What is the median of temperatures captured in April 2017?
# Q15.5 What is the maximum temperature of time slots with clear sky?


In [ ]:
# Using mask or .query syntax select rows/columns (float)

# Q16.1 What is the average temperature in celcius of the time slots with rainy_and_coudy=True?
# Q16.2 What is the average traffic volume on holidays?
# Q16.3 What is the average traffic volume on non-holidays?
# Q16.4 What is the average traffic volume in the highest quantile?
# Q16.5 What is the average traffic volume in the lowest quantile?


# 5. Basic date processing

You figure out that column date is to harsh for you, so you decided to convert it to a more plausible format:

- Use pandas method to_datetime() to convert the date to a good format.
- Extract year, month, day and weekday from your new date column. Save them to separate columns.
- How many columns has your data now?
- Drop column date, remember to set inplace parameter to True.

Hint: for datetime formatted date you could extract the year as follow:
```
data.date.dt.year
```
Very often date could be a ridiculously rich feature, sometimes it is holidays that matters, sometimes weekends, sometimes some special days like black friday.

Learn how to work with date in Python!


In [ ]:
# Create new columns based on `Captured` column

# Q17.1 Convert date to datetime format
# Q17.2 Extract and store `year`
# Q17.3 Extract and store `month`
# Q17.4 Extract and store `day`
# Q17.5 Extract and store `weekday` (Monday - 0, Sunday - 6)
# Q17.6 Extract and store `hour`


In [ ]:
# Find some date related information from the data (int)

# Q18.1 What is the weekday with the highest traffic volume?
# Q18.2 What is the weekday with the lowest traffic volume?
# Q18.3 What is the average traffic volume during months of September?
# Q18.4 What is the average traffic volume in the time period between 15-19 hours
# Q18.5 What is the average traffic volume on World Bicycle Day (June 3)?


# 6. Groupby

from the documentation https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

By “group by” we are referring to a process involving one or more of the following steps:

- Splitting the data into groups based on some criteria.
- Applying a function to each group independently.
- Combining the results into a data structure.

`.groupby()` is one of the most powerfull tool for feature engineering. Very often it is used to group object with the same categorical characteristics and compute some statistics (e.g. mean, max, etc.) of a their numerical characteric.

Instead of computing average traffic volume with for each month you could compute average traffic volumes for every month in a single command:
```
data.groupby('month')['traffic_volume'].mean()
```
You could also make multi-column groups:
```
data.groupby(['weekday','month'])['traffic_volume'].min()
```
next, you could compute multiple aggregation functions:
```
data.groupby(['weekday','month'])['traffic_volume'].agg([min, max])
```
instead of using built-in functions you could compute custom functions using apply:
```
import numpy as np
data.groupby(['weekday','month'])['traffic_volume'].apply(lambda x: np.quantile(x, .5))
```
and the coolest thing now is that you can map the results of groupby back on your DataFrame!
```
gp = data.groupby(['month'])['traffic_volume'].median()
data['gp_feature'] = data['month'].map(gp)
```
Now, if some timeslot has month == 2, its gp_feature will be equal to the median traffic volume amongst all observations in February

Read more examples in the documentation https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html


In [ ]:
# Create some groupby features

# Q19.1 `traffic_by_year` groupby `year` and compute median traffic volume.
# Q19.2 `traffic_by_weekday` groupby `weekday` and compute median traffic volume.
# Q19.3 `temperature_by_traffic` groupby `traffic_cat` and compute average temperature in celsius.



# Make sure your .ipynb is linearly executable     
# Kernel -> Restart & Run All -> No ERROR cells